System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 8 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 6 on 6 virtual cores
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 6


In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw3`


Status `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [31c24e10] Distributions v0.25.88
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


Load packages:

In [3]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

### Q1 Solution

According to the notations, we have: $\mathbf{y}_i \sim \mathcal{N}(\mathbf{X}_i\boldsymbol{\beta},\ \mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})$.

For a random vector $\mathbf{x}$ following a Multivariate Normal Distribution $\mathcal{N}(\boldsymbol{\mu},\ \boldsymbol{\Sigma})$, the probability density function is given by: $$f(\mathbf{x})=\frac{1}{\sqrt{(2\pi)^k\det(\boldsymbol{\Sigma})}}\exp\left(-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\mathbf{x}-\boldsymbol{\mu})\right)$$

So, we can plug in the parameters and get:
$$
\begin{aligned}
\mathcal{L}(\boldsymbol{\beta},\boldsymbol{\Sigma}, \sigma^2) &= \log\mathbf{P}(y_i|\boldsymbol{\beta},\boldsymbol{\Sigma}, \sigma^2)\\
&= \log\left(\frac{1}{\sqrt{(2\pi)^{n_i}\det(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})}}\exp\left(-\frac{1}{2}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})^{-1}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta})\right)\right)\\
&= -\frac{1}{2}\log\left((2\pi)^{n_i}\det(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})\right)
-\frac{1}{2}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})^{-1}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta}) \\
&= -\frac{1}{2}\log\left(\det(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})\right) 
-\frac{1}{2}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta})^T(\mathbf{Z}_i\boldsymbol{\Sigma}_{q\times q}\mathbf{Z}_i^T+\sigma^2\mathbf{I}_{n_i})^{-1}(\mathbf{y}_i-\mathbf{X}_i\boldsymbol{\beta})
-\frac{n_i}{2}\log\left(2\pi\right) \\
\end{aligned}
$$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [4]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_q, xtx, ztx, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

### Q2 Solution

In [ ]:
function logl!(
    obs :: LmmObs{T}, 
    β   :: Vector{T}, 
    L   :: Matrix{T}, 
    σ²  :: T) where T <: AbstractFloat

n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
# compute and return the log-likelihood

sleep(1e-3) # wait 1 ms as if your code takes 1ms

return zero(T)

end

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [ ]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [ ]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

Check that your answer matches that from Distributions.jl

In [ ]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [ ]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [ ]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

In [ ]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [ ]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [ ]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.